In [2]:
import numpy as np
from math import acos, sin, cos, tan
import pandas as pd

In [3]:
################## DATA FRAME ##################
datafile = '../Fullscale21.csv'
fields = ['Timestamp',
  'Roll', 'Pitch', 'Yaw',
  'LinearAccelNed X', 'LinearAccelNed Y', 'LinearAccelNed Z'
  ]

df = pd.read_csv(datafile, skipinitialspace=True, usecols=fields)

################## INIT VECTORS ##################
all_time = df['Timestamp'].values

tdata = all_time
tdata = tdata - all_time[0]

In [ ]:
def get_YPR_fixed_frame(bYPR):
    """Accepts body frame YPR, returns the FIXED FRAME YPR"""
    
    R = convert_YPR_fixed_frame(bYPR[0], bYPR[1], bYPR[2])

    # theta: 3x1 vector of YPR in the fixed frame
    theta = acosd(np.dot(R[:,2], [0, 0, 1]))
    
    return theta


def map_roll(x):
    return abs(np.mod(x - 180.0, 360.0) - 180.0)


def compare_change(theta_prev, theta_new, threshold=50):
    if type(threshold) is int or type(threshold) is float:
        # You have specified a single acceptable value, shared for all 3 axes
        roll_thresh, pitch_thresh, yaw_thresh = threshold, threshold, threshold
        individual = False
    elif len(threshold)==3:
        # You have given YPR separate thresholds
        roll_thresh, pitch_thresh, yaw_thresh = threshold[0], threshold[1], threshold[2]
        individual = True
    else:
        raise("TypeError: threshold parameter should either be an int or a 3x1 vector of ints")
    
    should_accept = True
    
    # Map the roll so it is triangular, peaks at 180. 0->0, 180->180, 360->0
    roll_prev = map_roll(theta_prev[0])
    roll_new = map_roll(theta_new[0])

    if individual:
        # DEALING WITH INDIVUDAL AXIS ROTATIONS
        # Roll
        if roll_prev - roll_new > roll_thresh:
            print("Too much ROLL detected.  Image should be discarded")
            return False
        # Pitch
        elif theta_prev[1] - theta_new[1] > pitch_thresh:
            print("Too much PITCH detected.  Image should be discarded")
            return False
        # Yaw
        elif theta_prev[2] - theta_new[2] > yaw_thresh:
            print("Too much YAW detected.  Image should be discarded")
            return False
        
    else:
        # DEALING WITH ALL AXES ROTATIONS (E.G. IN COMBINATION)
        theta_prev[0] = roll_prev
        theta_new[0] = roll_new
        
        dTheta = theta_new - theta_prev
        
        #if np.linalg.norm(dTheta) > threshold: 
        # ^Norm doesn't have a direct meaning like the difference in angle does
        if (dTheta > threshold).sum() > 0:
            return False

    return should_accept


# This builds the first orientation matrix
def init_orientation():
    bYPR = get_IMU_data()
    ffYPR = get_YPR_fixed_frame(bYPR)
    return ffYPR


# Put this before the incoming images
def judge_image(prev_orientation):
    # Collect incoming IMU data in real time
    bYPR = get_IMU_data()
    # Convert IMU data from body frame to fixed frame
    ffYPR = get_YPR_fixed_frame(bYPR)
    # Compare current thetas to previous orientation
    if compare_change(prev_orientation, ffYPR):
        return True, ffYPR
    else:
        return False, prev_orientation

In [4]:
def get_IMU_data(df, row_index):
    bYPR = np.array([df.iloc[3, row_index], df.iloc[2, row_index], df.iloc[1, row_index]])
    return bYPR

## Unit Testing